# Working with Google Sheets using Gspread

## Hello, I'm Anna

![career](images/career.png)

![py](images/py.png)

## Why on earth would I want to load data to Google Sheets?

**Possible use cases (1)**:

- Some users just can't live without their Excel files




![excel](images/dilbert_spreadsheet.gif)


**Possible use cases (2)**:

- "ETL" 

Combining data from multiple sources, cleaning and then loading to Google Sheet as a source for BI visualization tool (Tableau, etc.)


![etl](images/etl.png)

In [1]:
class Tweet(object):
    def __init__(self, embed_str=None):
        self.embed_str = embed_str

    def _repr_html_(self):
        return self.embed_str

s = ("""
<blockquote class="twitter-tweet" data-lang="en-gb"><p lang="en" dir="ltr">So if you&#39;re building out a quick CRUD app for e.g. internal use, Google Docs as a backend (consumed via JSON) is *surprisingly* powerful.</p>&mdash; Patrick McKenzie (@patio11) <a href="https://twitter.com/patio11/status/485239825314758656?ref_src=twsrc%5Etfw">5 July 2014</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
""")

**Possible use cases (3)**:

- "Database" (?)

In [2]:
Tweet(s)

## Importing the necessary libraries

We will use the following libraries:

- gspread – to interact with Google Spreadsheets (more info [here](https://gspread.readthedocs.io/en/latest/))
- gspread_dataframe – to allow data flow between Google Spreadsheet and a Pandas DataFrame (more info [here](https://pypi.org/project/gspread-dataframe/))
- oauth2client – to authorize the Google Drive API using OAuth 2.0 (more info [here](https://gspread.readthedocs.io/en/latest/oauth2.html))


**Installing the libraries with pip**:

```
pip install gspread
pip install gspread-dataframe
pip install oauth2client
```

In [6]:
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

## Getting the credentials



### Create project 

Go to https://console.developers.google.com/ and click "New Project"



![create_project](images/create_project.png)


### Enable APIs

Click "Enable APIs and Services"

![enable_apis](images/enable_api.jpg)




### Enable Google Sheets APIs

Search for Google Sheets API, then click "Enable"

![sheets_api](images/sheets_api.jpg)


### Create API credentials

Click "Create Credentials"

![create_cred](images/create_cred.jpg)


### Create Service Account Key

Choose "Service Account Key", assign Account Name and Account ID and select "JSON"

![key](images/key.jpg)


### Final steps

- Place the downloaded .json file with credentials in the folder with your project
- Make sure to share the spreadsheets with the created API account


![share](images/share.jpg)

## Connecting to Google Sheets

In [7]:
# adding drive scope will permit us create shpreadsheets

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']


# authorization process

credentials = ServiceAccountCredentials.from_json_keyfile_name('Gspread-eff37025c365.json', scope)
gc = gspread.authorize(credentials)

## Main functions

### Creating a Google Sheet

After creating a google sheet, we need to share it with our mail (otherwise it will be visible only to the API user)

In [5]:
sheet = gc.create('A new spreadsheet')
sheet.share('admi.fedotova@gmail.com', perm_type='user', role='writer')

### Opening Google Sheet

In [6]:
# by name
sheet = gc.open("test_gspread")
print(sheet)

<Spreadsheet 'test_gspread' id:18DWySEljDqg4jqByLFoS791C_qvd1YrMWnQVo42Ysa0>


In [8]:
# by id
sheet_id = '18DWySEljDqg4jqByLFoS791C_qvd1YrMWnQVo42Ysa0'
sheet = gc.open_by_key(sheet_id)
print(sheet)

<Spreadsheet 'test_gspread' id:18DWySEljDqg4jqByLFoS791C_qvd1YrMWnQVo42Ysa0>


### Selecting sheet's tabs

In [8]:
# getting a list of sheet's tabs
tabs = sheet.worksheets()
print(tabs)

[<Worksheet 'Sheet1' id:0>]


In [9]:
# selecting sheet's tab by name
tab_name = 'Sheet1'
tab = sheet.worksheet(tab_name)
print(tab)

<Worksheet 'Sheet1' id:0>


In [10]:
# selecting sheet's tab by position
tab = sheet.get_worksheet(0)
print(tab)

<Worksheet 'Sheet1' id:0>


### Updating sheet's cell

In [10]:
# by cell name
tab.update_acell('A1', 'Hello,')

# by cell index
tab.update_cell(1, 2, 'world!')

{'spreadsheetId': '18DWySEljDqg4jqByLFoS791C_qvd1YrMWnQVo42Ysa0',
 'updatedRange': 'Sheet1!B1',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

### Loading Pandas dataframe to Google Sheet

In [11]:
import numpy as np 
np.random.seed(0)

In [12]:
# creating a dataframe with random numbers
df = pd.DataFrame(np.random.randint(0,100,size=(25, 4)), columns=list('ABCD'))

# loading the dataframe into spreadsheet
set_with_dataframe(tab, df)

### Appending row to Google Sheet

In [13]:
row = [1, 2, 3, 4]
tab.append_row(row)

{'spreadsheetId': '18DWySEljDqg4jqByLFoS791C_qvd1YrMWnQVo42Ysa0',
 'tableRange': 'Sheet1!A1:D26',
 'updates': {'spreadsheetId': '18DWySEljDqg4jqByLFoS791C_qvd1YrMWnQVo42Ysa0',
  'updatedRange': 'Sheet1!A27:D27',
  'updatedRows': 1,
  'updatedColumns': 4,
  'updatedCells': 4}}

### Getting data from Google Sheet

In [15]:
# getting data from a specific cell
val = tab.acell('A2').value
print(val)

44


In [16]:
# getting data from a specific row
row_list = tab.row_values(1)
print(row_list)

['A', 'B', 'C', 'D']


In [17]:
# getting data from a specific column
col_list = tab.col_values(1)
print(col_list)

['A', '44', '67', '36', '88', '39', '81', '72', '69', '82', '29', '39', '57', '23', '28', '36', '17', '58', '41', '46', '14', '42', '6', '76', '20', '79', '1', '1']


### Loading data from Google Sheet to a Pandas dataframe

In [18]:
df2 = get_as_dataframe(tab, usecols=[0,1,2,3])
df2.head()

,A,B,C,D
0,44.0,47.0,64.0,67.0
1,67.0,9.0,83.0,21.0
2,36.0,87.0,70.0,88.0
3,88.0,12.0,58.0,65.0
4,39.0,87.0,46.0,88.0


## Case Study 


![case_etl](images/case_etl.png)

In [19]:
# loading the data into a dataframe
df = pd.read_csv('raw_data.csv')
df.head()

,sourceId,publisherId,countryName,os,lineCategory,conversions,earlyConversions,kpiGoalReach,has_goal,fraudIndex,appName
0,1940,1006950361451204608_100136,Brazil,Android,Entertainment,1,1,NaN,0,0.0,au.com.stklab.minehd
1,1880,75_112181.121819,Russia,iOS,Gambling,1,0,NaN,0,67.0,com.apalon.optimizer
2,2198,35_8168_10085,Republic of Korea,iOS,Games,3,0,250.0,1,0.0,com.lenovo.anyshare.gps
3,1548,61_sz36wfsa,United States,iOS,Finance,1,1,0.0,1,11.0,com.itube.samsung
4,2133,50_103_10053T_197369136878549,Mexico,Android,Food&Drinks,1,1,NaN,0,33.0,com.edujoy.masha.games


In [20]:
# opening spreadsheet by id
sheet_id = '1NElsAGUtKlT8c_a_q6wpMIAamgF1L-qhTIR8XbyUMV4'
sheet = gc.open_by_key(sheet_id)

# selecting spreadsheet tab
tab_name = 'raw'
tab = sheet.worksheet(tab_name)

# deleting existing content
tab.resize(rows=1)
tab.resize(rows=50000)

# loading the dataframe into spreadsheet
set_with_dataframe(tab, df)

### Cron Job

The last step was to schedule a cron job to execute the script once a week.

![cron](images/cron.png)

### My first code put into production 

(and it worked)

![magic](images/magic.gif)

## Key takeaways for python beginners


- There is a library for that
- Read the documentation and see examples
- Start small and iterate

## List of references:

- https://www.twilio.com/blog/2017/02/an-easy-way-to-read-and-write-to-a-google-spreadsheet-in-python.html
- https://gspread.readthedocs.io/en/latest/
- https://pypi.org/project/gspread-dataframe/
- http://www.indjango.com/access-google-sheets-in-python-using-gspread/
- https://medium.com/@CROSP/manage-google-spreadsheets-with-python-and-gspread-6530cc9f15d1

In [21]:
!jupyter nbconvert gspread.ipynb --to slides --post serve

[NbConvertApp] Converting notebook gspread.ipynb to slides
[NbConvertApp] Writing 314055 bytes to gspread.slides.html
[NbConvertApp] Redirecting reveal.js requests to https://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.5.0
Serving your slides at http://127.0.0.1:8000/gspread.slides.html
Use Control-C to stop this server
^C

Interrupted
